In [5]:
import json
import time

import requests
from rdflib import Graph, URIRef, Literal, Namespace
from tqdm import tqdm
from rdflib.namespace import RDF, SKOS,RDFS,SDO,OWL

In [6]:
g = Graph()
#space = "http://projet.fr/perso/"
api = "http://localhost/html/preprocessId.php?name="

errors = []
#g.bind("", "http://projet.fr/perso#")

#ns_perso_data = Namespace(space)
#ns_perso_schema = Namespace('"http://projet.fr/perso/"')
"""g.bind("skos", SKOS)
g.bind("owl", OWL)
g.bind("rdfs", RDFS)"""

#ns_film_data = Namespace('http://projet.fr/films_data/')
#ns_film_schema = Namespace('http://projet.fr/films_schema/')
ns_perso_schema = Namespace('http://projet.fr/perso_schema/')
ns_perso_data = Namespace('http://projet.fr/perso_data/')

g.bind('p_schema', ns_perso_schema)
g.bind("", ns_perso_data)


def getHero(k):
    url = api + str(k)
    reponse = requests.get(url)
    content = reponse.content
    y = json.loads(content)[0]
    return y

def getRace(y):
    race = y["appearance"]["race"]
    if race!="" and race!="-" and race!="null":
        raceNode = ns_perso_data[''.join(letter for letter in race.lower() if letter.isalnum())]
        g.add((raceNode, SKOS.prefLabel, Literal(race)))
        g.add((raceNode, RDF.type, SKOS.Concept))
        g.add((raceNode,RDFS.subClassOf,ns_perso_schema["Race"]))

def getRelatives(y):
    relatives = y["connections"]["relatives"]
    for relative in relatives:
        relativeNode = ns_perso_data['P' + ''.join(letter for letter in relative.lower() if letter.isalnum())]
        g.add((relativeNode, SDO.name, Literal(relative)))
        g.add((relativeNode, RDF.type, ns_perso_schema["Character"]))
        g.add((relativeNode, RDF.type, SDO.Person))



def getHeroNode(y):
    superName = y["name"]
    name = y["biography"]["full-name"]
    id = name.lower() + superName.lower()
    heroNode = ns_perso_data['H'+''.join(letter for letter in id  if letter.isalnum())]
    conceptNode = ns_perso_data['C'+''.join(letter for letter in superName if letter.isalnum())]
    persoNode = ns_perso_data['P'+''.join(letter for letter in id if letter.isalnum())]
    g.add((heroNode, SDO.name, Literal(superName)))
    g.add((conceptNode, RDFS.subClassOf, ns_perso_schema["Character"]))
    g.add((conceptNode, RDF.type, SKOS.Concept))
    g.add((conceptNode, SKOS.prefLabel, Literal(superName)))
    if name != "":
        g.add((persoNode, SDO.name, Literal(name)))
    g.add((persoNode, RDF.type, ns_perso_schema["Character"]))
    g.add((persoNode, RDF.type, ns_perso_schema["Fictional"]))
    g.add((persoNode, RDF.type, SDO.Person))
    g.add((heroNode, RDF.type, ns_perso_schema["Character"]))
    g.add((heroNode, RDF.type, ns_perso_schema["Fictional"]))
    g.add((heroNode, RDF.type, SDO.Person))


def getPublisher(y):
    publisher = y["biography"]["publisher"]
    publisherNode = ns_perso_data[''.join(letter for letter in publisher.lower() if letter.isalnum())]
    g.add((publisherNode, RDF.type, SKOS.Concept))
    g.add((publisherNode, RDFS.subClassOf, ns_perso_schema['Publisher']))
    g.add((publisherNode, SKOS.prefLabel, Literal(publisher)))


def getColor(y):
    colorsEye = y["appearance"]["eye-color"]
    colorsHair = y["appearance"]["hair-color"]
    for colorEye in colorsEye:
        eyeNode = ns_perso_data[''.join(letter for letter in colorEye if letter.isalnum())]
        g.add((eyeNode, SKOS.prefLabel, Literal(colorEye)))
        g.add((eyeNode, RDFS.subClassOf, ns_perso_schema['Color']))
        g.add((eyeNode, RDF.type, SKOS.Concept))
    for colorHair in colorsHair:
        hairNode = ns_perso_data[''.join(letter for letter in colorHair if letter.isalnum())]
        g.add((hairNode, RDF.type, SKOS.Concept))
        g.add((hairNode, SKOS.prefLabel, Literal(colorHair)))
        g.add((hairNode, RDFS.subClassOf, ns_perso_schema['Color']))


def getOccupation(y):
    occupations = y["work"]["occupation"]
    for occupation in occupations:
        occupationNode = ns_perso_data[''.join(letter for letter in occupation if letter.isalnum())]
        g.add((occupationNode, RDF.type, SKOS.Concept))
        g.add((occupationNode, SKOS.prefLabel, Literal(occupation)))
        g.add((occupationNode, RDFS.subClassOf, ns_perso_schema['Occupation']))





def createNodes(k):
    try:   
        y = getHero(k)
    except Exception as e:
        print(e)
        errors.append(e)
    else:
        getHeroNode(y)
        getOccupation(y)
        getColor(y)
        getRace(y)




In [7]:
for k in tqdm(range(1, 732)):
    createNodes(k)
    time.sleep(0.1)

g.serialize(destination="database.ttl", format="turtle")

  0%|          | 0/731 [00:00<?, ?it/s]

 38%|███▊      | 279/731 [04:54<08:06,  1.08s/it]

Expecting value: line 1 column 1 (char 0)


 67%|██████▋   | 491/731 [08:37<04:28,  1.12s/it]

Expecting value: line 1 column 1 (char 0)


100%|██████████| 731/731 [12:49<00:00,  1.05s/it]


<Graph identifier=Nd98fff1b7bcf4f3fad1f346aefd5f3d0 (<class 'rdflib.graph.Graph'>)>